In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Vehicle_Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//tr[@class="detail-app-row"]')
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                while True: 
                    for tr in list_tr:
                        make = tr.xpath('./td[2]/text()')[0].strip()
                        model = tr.xpath('./td[3]/text()')[0].strip()
                        engine = tr.xpath('./td[4]/text()')[0].strip()
                        year = int(tr.xpath('./td[1]/text()')[0].strip())

                        if make not in dict_vehicle:
                            dict_vehicle[make] = {model: {engine: [year]}}
                        else:
                            if model not in dict_vehicle[make]:
                                dict_vehicle[make][model] = {engine: [year]}
                            else:
                                if engine not in dict_vehicle[make][model]:
                                    dict_vehicle[make][model][engine] = [year]
                                else:
                                    dict_vehicle[make][model][engine].append(year)

                        dict_vehicle[make][model][engine] = list(set(dict_vehicle[make][model][engine]))
                        dict_vehicle[make][model][engine].sort()
                        
                    if len(html.xpath('//a[@aria-label="Next"]')) == 0:
                        break
                    else:
                        resp = requests.get('https://www.dormanproducts.com' + html.xpath('//a[@aria-label="Next"]/@href')[0].strip(),
                                            headers=UA.get_User_Agent_Requests(),
                                            proxies=Proxy.get_Proxy_Requests()).text
                        
                        soup = BeautifulSoup(resp, 'lxml')
                        html = etree.HTML(str(soup))
                        
                        list_tr = html.xpath('//tr[@class="detail-app-row"]')
    
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_year = []
                        for engine in dict_vehicle[make][model]:
                            list_year += dict_vehicle[make][model][engine]
                        list_year = list(set(list_year))
                        list_year.sort()

                        year_min = list_year[0]
                        year_max = list_year[-1]
                        if year_min == year_max:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                list_vehicle.sort()
                
                vehicle_1 = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]:
                            year_min = dict_vehicle[make][model][engine][0]
                            year_max = dict_vehicle[make][model][engine][-1]
                            if year_min == year_max:
                                if engine == '':
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                            else:
                                if engine == '':
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                list_vehicle.sort()
                
                vehicle_2 = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Vehicle_Url': input_.loc[a, 'Vehicle_Url'],
                                         'Vehicle_1': vehicle_1,
                                         'Vehicle_2': vehicle_2}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Vehicle_Url': input_.loc[a, 'Vehicle_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Vehicle_Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Vehicle_Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Vehicle_Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=52616&PartType=Fuel%20Filler%20Neck
[尝试次数：1] - [剩余数量：0] - [当前时间：15:40:51]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73537&PartType=Fuel%20Filler%20Neck
[尝试次数：1] - [剩余数量：0] - [当前时间：15:41:01]

输出ing...

Done ~
